In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt
import plotly.express as px
url_carpeta='C:/Users/Bluetab/Documents/Universidad/Archivos 2doSemestre/Visualizacion/'
credit = pd.read_csv(url_carpeta+'credit_record.csv')  
application = pd.read_csv(url_carpeta+'application_record.csv') 
credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [ ]:
len(set(application['ID'])) # ¿cuántos ID únicos en el registro de la aplicación?

In [ ]:
len(set(credit['ID'])) # ¿cuántos ID únicos en el registro de crédito?

In [ ]:
len(set(application['ID']).intersection(set(credit['ID']))) # ¿cuántos de los IDs comparten dos tablas?

# KPI

In [4]:
grouped = credit.groupby('ID')
### convertir datos de crédito a un formato amplio que cada ID es una fila
pivot_tb = credit.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')
pivot_tb['open_month'] = grouped['MONTHS_BALANCE'].min() # mes de apertura
pivot_tb['end_month'] = grouped['MONTHS_BALANCE'].max() # mes final
pivot_tb['ID'] = pivot_tb.index
pivot_tb = pivot_tb[['ID', 'open_month', 'end_month']]
pivot_tb['window'] = pivot_tb['end_month'] - pivot_tb['open_month']
pivot_tb.reset_index(drop = True, inplace = True)
credit = pd.merge(credit, pivot_tb, on = 'ID', how = 'left')
credit0 = credit.copy()
credit = credit[credit['window'] > 20] # eliminar a los usuarios cuya ventana de observación menos de 20
credit['status'] = np.where((credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 1, 0) # analyze > 60 days past due 
credit['status'] = credit['status'].astype(np.int8) # 1: overdue 0: not
credit['month_on_book'] = credit['MONTHS_BALANCE'] - credit['open_month'] # calculate month on book: how many months after opening account
credit.sort_values(by = ['ID','month_on_book'], inplace = True)

##### 
denominator = pivot_tb.groupby(['open_month']).agg({'ID': ['count']}) # count how many users in every month the account was opened
denominator.reset_index(inplace = True)
denominator.columns = ['open_month','sta_sum']

#####
kpi = credit.groupby(['open_month','month_on_book']).agg({'ID': ['count']}) 
kpi.reset_index(inplace = True)
kpi.columns = ['open_month','month_on_book','sta_sum'] 
kpi['due_count'] = np.nan
kpi = kpi[['open_month','month_on_book','due_count']] # delete aggerate column
kpi = pd.merge(kpi, denominator, on = ['open_month'], how = 'left') # join sta_sum colun to kpi table
kpi

,open_month,month_on_book,due_count,sta_sum
0,-60,0,NaN,415
1,-60,1,NaN,415
2,-60,2,NaN,415
3,-60,3,NaN,415
4,-60,4,NaN,415
...,...,...,...,...
1655,-21,17,NaN,858
1656,-21,18,NaN,858
1657,-21,19,NaN,858
1658,-21,20,NaN,858


In [ ]:
for j in range(-60,1): # outer loop: month in which account was opened
    ls = []
    for i in range(0,61): # inner loop time after the credit card was granted
        due = list(credit[(credit['status'] == 1) & (credit['month_on_book'] == i) & (credit['open_month'] == j)]['ID']) # get ID which satisfy the condition
        ls.extend(due) # As time goes, add bad customers
        kpi.loc[(kpi['month_on_book'] == i) & (kpi['open_month'] == j), 'due_count'] = len(set(ls)) # calculate non-duplicate ID numbers using set()
        
kpi['sta_rate']  = kpi['due_count'] / kpi['sta_sum'] # calculate cumulative % of bad customers
kpi        

Using `pivot` to convert long data to wide data:

In [ ]:
kpi_wide = kpi.pivot(index = 'open_month',
                             columns = 'month_on_book',
                             values = 'sta_rate')
kpi_wide

Esta es una tabla de análisis de kpi. Las filas representan meses de apertura de cuentas, las columnas representan meses después de la apertura de cuentas y los valores son la tasa acumulada de vencimiento. A medida que el mes abierto se cierra a 0, la ventana de observación se acorta, por lo que el campo triangular de la mitad inferior se llena con nan. 

In [ ]:
kpi0 = kpi_wide.replace(0,np.nan)
lst = [i for i in range(0,61)]
kpi_wide[lst].T.plot(legend = False, grid = True, title = 'Porcentaje acumulado de malos clientes (> 60 días de atraso) ')
#plt.axvline(30)
#plt.axvline(25)
#plt.axvline(20)
plt.xlabel('Historial')
plt.ylabel('% Acumulado> 60 días de vencimiento ')
plt.show()

# Observar el análisis de la ventana

Debido a dos razones, la cancelación de la cuenta y la cancelación, nuestra observación de las cuentas se truncará. La ventana de observación es un parámetro importante a considerar. Si la ventana de observación es demasiado corta, el comportamiento de los usuarios no se mostrará por completo, lo que traerá un ruido innecesario a nuestros datos.

Para observar cuántas cuentas aumentan a medida que se extiende la ventana de observación, raelicemos la siguiente grafica.  

In [ ]:
lst = []
for i in range(0,61):
    ratio = len(pivot_tb[pivot_tb['window'] < i]) / len(set(pivot_tb['ID']))
    lst.append(ratio)
    
pd.Series(lst).plot(legend = False, grid = True, title = ' ')
plt.xlabel('Ventana de Observacion')
plt.ylabel('Relacion')
plt.show()

Pudimos ver que una ventana de observación de 60 meses cubre a todos los aplicadores, mientras que la ventana de 20 meses contiene registros de aproximadamente el 52%. 

In [5]:
def calculate_observe(credit, command):
    '''calculate observe window
    '''
    id_sum = len(set(pivot_tb['ID']))
    credit['status'] = 0
    exec(command)
    #credit.loc[(credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1
    credit['month_on_book'] = credit['MONTHS_BALANCE'] - credit['open_month']
    minagg = credit[credit['status'] == 1].groupby('ID')['month_on_book'].min()
    minagg = pd.DataFrame(minagg)
    minagg['ID'] = minagg.index
    obslst = pd.DataFrame({'month_on_book':range(0,61), 'rate': None})
    lst = []
    for i in range(0,61):
        due = list(minagg[minagg['month_on_book']  == i]['ID'])
        lst.extend(due)
        obslst.loc[obslst['month_on_book'] == i, 'rate'] = len(set(lst)) / id_sum 
    return obslst['rate']

command = "credit.loc[(credit['STATUS'] == '0') | (credit['STATUS'] == '1') | (credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"   
morethan1 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '1') | (credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"   
morethan30 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan60 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan90 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan120 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '5'), 'status'] = 1"
morethan150 = calculate_observe(credit, command)

In [6]:
obslst = pd.DataFrame({'vencidos más de 30 días': morethan30,
                       'vencidos más de 60 días': morethan60,
                       'vencidos más de 90 días': morethan90,
                       'vencidos más de 120 días': morethan120,
                       'vencidos más de 150 días': morethan150
                        })

In [7]:
fig_kpi=px.line(obslst,title='Analisis % de malos clientes',labels=dict(index="Historial (meses)", value="Acumulativo (%)", variable="Categoria vencidos"))

Esta gráfica podría verse como una versión promedio (a lo largo de un mes abierto) de la gráfica del kpi.
Para una fecha de vencimiento más larga, necesita una ventana de observación más larga. Por ejemplo, más de 150 días de atraso necesita al menos 5 meses hasta que aparezca el primer * mal cliente *. Para la mayoría de las situaciones, una ventana de observación de 20 meses podría cubrir a la mayoría de los *clientes malos*. Sin embargo, durante los 30 días de vencimiento, pudimos ver que después de 30 meses en el historial, todavía hay nuevos * malos clientes * que se unen a la lista. Por lo tanto, una ventana de observación de 20 meses en el Historial será apropiada. Aquellos que existan más cortos que la ventana de observación deben ser excluidos de nuestro análisis, por lo que podría ver que eliminé usuarios cuya ventana de observación sea menor a 20 en la última sección (la ventana podría cambiarse). 